In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import scipy.stats as stats
import math

!pip install sklearn
os.getcwd()

You should consider upgrading via the '/homes/iws/bhimar/cse481ds-mental-health/venv/bin/python3 -m pip install --upgrade pip' command.


'/homes/iws/bhimar/cse481ds-mental-health/notebooks/county_level/Regression Analysis'

In [2]:
df = pd.read_csv('../../../data/processed_data/full_df.csv')
df.head()

,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,...,April Average Precipitation,May Average Precipitation,June Average Precipitation,July Average Precipitation,August Average Precipitation,September Average Precipitation,October Average Precipitation,November Average Precipitation,December Average Precipitation,RUCC
0,1,1,1001,AL,Autauga County,2010,1.0,9778.1,670.0,137881.0,...,2.17,5.72,2.47,3.38,4.38,1.38,3.14,5.01,1.89,2.0
1,1,3,1003,AL,Baldwin County,2010,1.0,8221.7,2148.0,449589.0,...,2.31,6.31,4.94,4.08,9.38,2.24,2.06,6.11,2.19,3.0
2,1,9,1009,AL,Blount County,2010,1.0,8929.7,787.0,155580.0,...,5.66,7.47,3.39,3.55,4.31,1.23,3.12,4.62,2.07,1.0
3,1,15,1015,AL,Calhoun County,2010,1.0,11526.2,2035.0,313874.0,...,3.73,6.46,2.70,4.00,3.78,1.55,2.96,3.06,2.24,3.0
4,1,31,1031,AL,Coffee County,2010,1.0,9004.7,645.0,126516.0,...,1.88,4.71,3.84,2.78,6.30,2.41,1.38,3.71,2.03,4.0


## Preprocess Data

### Filter out NaN cols and rows

In [3]:
identifying_fields = ['State Abbreviation', 'Name', 'Release Year']
with open("factors.txt") as f:
     factors = f.read().splitlines()
with open("outcomes.txt") as f:
     outcomes = f.read().splitlines()
filtered_df = df[identifying_fields + factors + outcomes]
print(filtered_df.shape[0])
filtered_df.head(n=20)
for col in factors + outcomes:
    print(col + "-->" + str(filtered_df[col].isna().sum()))
print()
filtered_df = filtered_df.dropna()
print(filtered_df.shape[0])
for col in factors + outcomes:
    print(col + "-->" + str(filtered_df[col].isna().sum()))

6280
High school graduation raw value-->33
Unemployment raw value-->0
Some college raw value-->844
Ratio of population to mental health providers-->1758
Median household income raw value-->844
Average Temperature-->0
January Average Temperature-->0
February Average Temperature-->0
March Average Temperature-->0
April Average Temperature-->0
May Average Temperature-->0
June Average Temperature-->0
July Average Temperature-->0
August Average Temperature-->0
September Average Temperature-->0
October Average Temperature-->0
November Average Temperautre-->0
December Average Temperature-->0
Average Precipitation-->0
January Average Precipitation-->0
February Average Precipitation-->0
March Average Precipitation-->0
April Average Precipitation-->0
May Average Precipitation-->0
June Average Precipitation-->0
July Average Precipitation-->0
August Average Precipitation-->0
September Average Precipitation-->0
October Average Precipitation-->0
November Average Precipitation-->0
December Average Pre

### Split into Train and Test

In [4]:
from sklearn.utils import shuffle
filtered_df = shuffle(filtered_df)
X_train = filtered_df[factors][:math.ceil(filtered_df.shape[0] * 0.8)]
X_test = filtered_df[factors][-1 * math.floor(filtered_df.shape[0] * 0.2):]
print(filtered_df.shape[0])
print(X_train.shape[0])
print(X_test.shape[0])
print(pd.merge(X_train, X_test)) # expect likely empty
print()

Y_train = filtered_df[outcomes][:math.ceil(filtered_df.shape[0] * 0.8)]
Y_test = filtered_df[outcomes][-1 * math.floor(filtered_df.shape[0] * 0.2):]
print(filtered_df.shape[0])
print(Y_train.shape[0])
print(Y_test.shape[0])
print()

4492
3594
898
Empty DataFrame
Columns: [High school graduation raw value, Unemployment raw value, Some college raw value, Ratio of population to mental health providers, Median household income raw value, Average Temperature, January Average Temperature, February Average Temperature, March Average Temperature, April Average Temperature, May Average Temperature, June Average Temperature, July Average Temperature, August Average Temperature, September Average Temperature, October Average Temperature, November Average Temperautre, December Average Temperature, Average Precipitation, January Average Precipitation, February Average Precipitation, March Average Precipitation, April Average Precipitation, May Average Precipitation, June Average Precipitation, July Average Precipitation, August Average Precipitation, September Average Precipitation, October Average Precipitation, November Average Precipitation, December Average Precipitation, RUCC]
Index: []

[0 rows x 32 columns]

4492
3594
8

## Lasso Regression

In [5]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_error

### Lasso regression for predicting Poor Mental Health Days

In [6]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
model = LassoCV(cv=cv, n_jobs=-1)
model.fit(X_train, Y_train.iloc[:,0])
print('alpha: %f' % model.alpha_)

alpha: 4.198674


In [7]:
predictions = model.predict(X_test)
mae = mean_absolute_error(Y_test.iloc[:,0], predictions)
print(mae)

0.42865603954865117


In [8]:
coeffs = model.coef_
important_variables = [i for i, e in enumerate(coeffs) if e != 0]
for i in important_variables:
    print(factors[i])
    print(coeffs[i])

Ratio of population to mental health providers
2.2717624311817942e-06
Median household income raw value
-2.42721543472985e-05
